In [1]:
from decouple import config
import intrinio_sdk
from intrinio_sdk.rest import ApiException
import numpy as np 
import pandas as pd
import re
import string

In [2]:
intrinio_sdk.ApiClient().configuration.api_key['api_key'] = config('INTRINIO_KEY')

security_api = intrinio_sdk.SecurityApi()

In [3]:
capi = intrinio_sdk.CompanyApi()

In [86]:
# Returns Dataframe of fundamental indicators 

def fundamentals_to_df(tkr_id, n_toget):
    intrinio_sdk.ApiClient().configuration.api_key['api_key'] = config('INTRINIO_KEY')

    security_api = intrinio_sdk.SecurityApi()
    
    capi = intrinio_sdk.CompanyApi()
    fapi = intrinio_sdk.FundamentalsApi()

    fund_params = {
        'identifier'     :tkr_id,
        'filed_after'    :'',
        'filed_before'   :'',
        'reported_only'  : False,
    #     'fiscal_year'  :fiscal_year,
        'statement_code' :'',
        'type'           :'',
        'start_date'     :'',
        'end_date'       :'',
        'page_size'      :n_toget,
        'next_page'      :''
    }
    
    fundamentals = capi.get_company_fundamentals(**fund_params)
    id_dict = {
        "start_date"     : [],
        "end_date"       : [],
        "fiscal_year"    : [],
        "fiscal_period"  : [],
        "id"             : [],
        "statement_code" : [],
    }

    for i in np.arange(len(fundamentals.fundamentals)):
        id_dict['start_date'].append(fundamentals.fundamentals[i].start_date)
        id_dict['end_date'].append(fundamentals.fundamentals[i].end_date) 
        id_dict['fiscal_year'].append(fundamentals.fundamentals[i].fiscal_year)
        id_dict['fiscal_period'].append(fundamentals.fundamentals[i].fiscal_period)
        id_dict['id'].append(fundamentals.fundamentals[i].id)
        id_dict['statement_code'].append(fundamentals.fundamentals[i].statement_code)

    id_df = pd.DataFrame.from_dict(id_dict)
    
    qtrs = ['Q1', 'Q2', 'Q3', 'Q4']
    income_statements = (id_df.loc[(id_df['statement_code'] == 'income_statement') &
                                   (id_df['fiscal_period'].isin(qtrs)==True)]
                         .sort_values(by='start_date'))
    
    fund_dict = {
            'date'             : [],
            'quarter'          : [],
            'fundamental_name' : [],
            'value'            : [],
        }
    for row in income_statements.iterrows():
        fund_get = fapi.get_fundamental_reported_financials(row[1]['id'])
        f_got = fund_get.fundamental_dict
        fins = fund_get.reported_financials_dict
        for f in fins:
            fund_dict['date'].append(str(f_got['filing_date']).split(' ')[0])
            fund_dict['quarter'].append(f_got['fiscal_period'])
            fund_dict['fundamental_name'].append(f['xbrl_tag']['name'].lower())
            fund_dict['value'].append(f['value'])
            
    return_df = pd.DataFrame.from_dict(fund_dict)
    return(return_df)



In [89]:
#Returns fundamentals in dictionary form


def get_fundamentals(tkr_id, n_toget, sandbox=False):
    if sandbox == False:
        intrinio_sdk.ApiClient().configuration.api_key['api_key'] = config('INTRINIO_KEY')

        security_api = intrinio_sdk.SecurityApi()
    elif sandbox == True:
        intrinio_sdk.ApiClient().configuration.api_key['api_key'] = config('INTRINIO_SANDBOX_KEY')

        security_api = intrinio_sdk.SecurityApi()

    
    capi = intrinio_sdk.CompanyApi()
    fapi = intrinio_sdk.FundamentalsApi()

    fund_params = {
        'identifier'     :tkr_id,
        'filed_after'    :'',
        'filed_before'   :'',
        'reported_only'  : False,
    #     'fiscal_year'  :fiscal_year,
        'statement_code' :'',
        'type'           :'',
        'start_date'     :'',
        'end_date'       :'',
        'page_size'      :n_toget,
        'next_page'      :''
    }
    
    fundamentals = capi.get_company_fundamentals(**fund_params)
    id_dict = {
        "start_date"     : [],
        "end_date"       : [],
        "fiscal_year"    : [],
        "fiscal_period"  : [],
        "id"             : [],
        "statement_code" : [],
    }

    for i in np.arange(len(fundamentals.fundamentals)):
        id_dict['start_date'].append(fundamentals.fundamentals[i].start_date)
        id_dict['end_date'].append(fundamentals.fundamentals[i].end_date) 
        id_dict['fiscal_year'].append(fundamentals.fundamentals[i].fiscal_year)
        id_dict['fiscal_period'].append(fundamentals.fundamentals[i].fiscal_period)
        id_dict['id'].append(fundamentals.fundamentals[i].id)
        id_dict['statement_code'].append(fundamentals.fundamentals[i].statement_code)

    id_df = pd.DataFrame.from_dict(id_dict)
    
    qtrs = ['Q1', 'Q2', 'Q3', 'Q4']
    income_statements = (id_df.loc[(id_df['statement_code'] == 'income_statement') &
                                   (id_df['fiscal_period'].isin(qtrs)==True)]
                         .sort_values(by='start_date'))
    
    fund_dict = {}
    for row in income_statements.iterrows():
        fund_get = fapi.get_fundamental_reported_financials(row[1]['id']).reported_financials_dict
        funds = {}
        funds['start_date'] = row[1]['start_date']
        funds['end_date'] = row[1]['end_date']
        funds['fiscal_year'] = row[1]['fiscal_year']
        funds['quarter'] = row[1]['fiscal_period']
        for f in fund_get:
            fs = {
                'unit':f['xbrl_tag']['unit'],
                'value':f['value']
            }
            funds[f['xbrl_tag']['name'].lower()] = fs
        fund_dict[row[1]['id']] = funds
    return(fund_dict)

In [5]:
# This function will return a dictionary of company names and symbol
# or abbreviation. Don't run this on non-sandbox api unless you
# want a huge list.

def get_company_info(id_type=''):
    """ticker, name, lei, id, cik"""
    capi = intrinio_sdk.CompanyApi()
    companies = capi.get_all_companies().companies_dict
    names_dict = {}
    for company in companies:
        names_dict[company['name']] = company[id_type]
    return(names_dict)

In [25]:
fund_test = get_fundamentals('AAPL', 500)

In [23]:
for k in fund_test.keys():
    print('\n', k, '\n')
    for f in fund_test[k]:
        print(f, fund_test[k][f])
    print('\n')


 fun_RzmlOX 

start_date 2006-10-01
end_date 2007-09-29
fiscal_year 2007.0
quarter FY
net sales {'unit': 'usd', 'value': 24578000000.0}
cost of sales {'unit': 'usd', 'value': 16426000000.0}
gross margin {'unit': 'usd', 'value': 8152000000.0}
research and development {'unit': 'usd', 'value': 782000000.0}
selling, general and administrative {'unit': 'usd', 'value': 2963000000.0}
total operating expenses {'unit': 'usd', 'value': 3745000000.0}
operating income {'unit': 'usd', 'value': 4407000000.0}
other income and expense {'unit': 'usd', 'value': 599000000.0}
income before provision for income taxes {'unit': 'usd', 'value': 5006000000.0}
provision for income taxes {'unit': 'usd', 'value': 1511000000.0}
net income {'unit': 'usd', 'value': 3495000000.0}
basic {'unit': 'shares', 'value': 864595000.0}
diluted {'unit': 'shares', 'value': 889292000.0}



 fun_EgArPX 

start_date 2007-09-30
end_date 2008-09-27
fiscal_year 2008.0
quarter FY
net sales {'unit': 'usd', 'value': 37491000000.0}
cost 

In [ ]:
# Make list, and dict of company names.
# Run this if you want them all

names_dict = get_company_info('ticker')
names_list = list(names_dict.values())
tkr_dict = {v: k for k, v in names_dict.items()}

In [27]:
# 30 test companies (same as sandbox)

names_dict = {'Apple Inc': 'AAPL',
 'Exxon Mobil Corp': 'XOM',
 'Coca-Cola Co': 'KO',
 'Intel Corp': 'INTC',
 'Walmart Inc': 'WMT',
 'Microsoft Corp': 'MSFT',
 'International Business Machines Corp': 'IBM',
 'Chevron Corp': 'CVX',
 'Johnson & Johnson': 'JNJ',
 'Walt Disney Co, The': 'DIS',
 'Procter & Gamble Co': 'PG',
 'Pfizer Inc': 'PFE',
 'Verizon Communications Inc': 'VZ',
 'Boeing Co': 'BA',
 'Merck & Co Inc': 'MRK',
 'Cisco Systems Inc': 'CSCO',
 'Home Depot Inc': 'HD',
 "McDonald's Corp": 'MCD',
 '3M Co': 'MMM',
 'General Electric Co': 'GE',
 'United Technologies Corp': 'UTX',
 'Nike Inc': 'NKE',
 'Caterpillar Inc': 'CAT',
 'Visa Inc': 'V',
 'JPMorgan Chase & Co': 'JPM',
 'American Express Co': 'AXP',
 'Goldman Sachs Group Inc': 'GS',
 'UnitedHealth Group Inc': 'UNH',
 'Travelers Companies Inc': 'TRV',
 'DowDuPont Inc': 'DWDP'}

tkr_dict = {v: k for k, v in names_dict.items()}
names_list = list(names_dict.values())


In [ ]:
# List of all the different indicators listed by companies on names_list

reported_types = []

for t in names_list:
    t_fun = get_fundamentals(t, 100)
    common = ['start_date', 'end_date', 'fiscal_year', 'quarter']
    for k in t_fun.keys():
        for t in t_fun[k].keys():
            if t not in common:
                if t not in reported_types:
                    reported_types.append(t)
    
    

In [ ]:
# count them

len(reported_types)

In [88]:
# Prints pretty report of gross margin or lack there of 

for t in names_list:
    t_fun = get_fundamentals(t, 100)
    for k in t_fun.keys():
        if 'gross margin' in t_fun[k].keys():
            print('Company: ', tkr_dict[t])
            print('Ticker: ', t)
            print('Fiscal Year: ', t_fun[k]['fiscal_year'])
            print('Quarter: ', t_fun[k]['quarter'])
            print('Gross Margin: ', t_fun[k]['gross margin'])
        else:
            print(tkr_dict[t], ' reports no gross margin')
            print('for ', t_fun[k]['fiscal_year'], ' ', t_fun[k]['quarter'])

Company:  Apple Inc
Ticker:  AAPL
Fiscal Year:  2017.0
Quarter:  Q1
Gross Margin:  {'unit': 'usd', 'value': 30176000000.0}
Company:  Apple Inc
Ticker:  AAPL
Fiscal Year:  2017.0
Quarter:  Q2
Gross Margin:  {'unit': 'usd', 'value': 20591000000.0}
Company:  Apple Inc
Ticker:  AAPL
Fiscal Year:  2017.0
Quarter:  Q3
Gross Margin:  {'unit': 'usd', 'value': 17488000000.0}
Apple Inc  reports no gross margin
for  2017.0   Q4
Company:  Apple Inc
Ticker:  AAPL
Fiscal Year:  2018.0
Quarter:  Q1
Gross Margin:  {'unit': 'usd', 'value': 33912000000.0}
Company:  Apple Inc
Ticker:  AAPL
Fiscal Year:  2018.0
Quarter:  Q2
Gross Margin:  {'unit': 'usd', 'value': 23422000000.0}
Company:  Apple Inc
Ticker:  AAPL
Fiscal Year:  2018.0
Quarter:  Q3
Gross Margin:  {'unit': 'usd', 'value': 20421000000.0}
Apple Inc  reports no gross margin
for  2018.0   Q4
Company:  Apple Inc
Ticker:  AAPL
Fiscal Year:  2019.0
Quarter:  Q1
Gross Margin:  {'unit': 'usd', 'value': 32031000000.0}
Company:  Apple Inc
Ticker:  AAPL
F

ApiException: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 14 Aug 2019 23:21:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '239', 'Connection': 'keep-alive', 'Vary': 'Origin,Accept-Encoding'})
HTTP response body: {"error":"1 Minute Call Limit Reached","message":"You have exceeded the limits for how frequently you can API calls for the access code 'com_fin_data'. Please contact support if you need a higher limit.","access_codes":["us_fundamentals"]}


In [ ]:
get_company_info('ticker')

In [30]:
import time

In [37]:
# If you want to use get_fundamentals() in a loop, do something like this
# to avoid limits.

types_list = []
i = 0
for t in names_list:
    i += 1
    print(i)
    time.sleep(10)
    if i > 10:
        print('sleep')
        time.sleep(60)
        i = 0
    t_fun = get_fundamentals(t, 100)
    common = ['start_date', 'end_date', 'fiscal_year', 'quarter']
    for k in t_fun.keys():
        types_q = t_fun[k].keys()
        types_filtered = []
        if len(types_q) != 0:
            for t in types_q:
                if t not in common: 
                    types_filtered.append(t)
        types_list.append(types_filtered)

1
2
3
4
5
6
7
8
9
10
11
sleep
1
2
3
4
5
6
7
8
9
10
11
sleep
1
2
3
4
5
6
7
8


In [71]:
fapi.get_fundamental_reported_financials('fun_zE2eBo').fundamental_dict

{'id': 'fun_zE2eBo',
 'statement_code': 'income_statement',
 'fiscal_year': 2019.0,
 'fiscal_period': 'Q3',
 'type': 'reported',
 'start_date': datetime.date(2019, 3, 31),
 'end_date': datetime.date(2019, 6, 29),
 'filing_date': datetime.datetime(2019, 7, 31, 16, 31, 53, tzinfo=tzutc()),
 'company': {'id': 'com_NX6GzO',
  'ticker': 'AAPL',
  'name': 'Apple Inc',
  'lei': 'HWUPKR0MPOU8FGXBT394',
  'cik': '0000320193'}}

In [54]:
fapi = intrinio_sdk.FundamentalsApi()

In [87]:
fundamentals_to_df('AAPL', 100).head()

,date,quarter,fundamental_name,value
0,2018-02-02,Q1,net sales,7.835100e+10
1,2018-02-02,Q1,cost of sales,4.817500e+10
2,2018-02-02,Q1,gross margin,3.017600e+10
3,2018-02-02,Q1,research and development,2.871000e+09
4,2018-02-02,Q1,"selling, general and administrative",3.946000e+09


In [66]:
dir(fapi.get_fundamental_reported_financials('fun_zE2eBo'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_fundamental',
 '_next_page',
 '_reported_financials',
 'attribute_map',
 'discriminator',
 'fundamental',
 'fundamental_dict',
 'next_page',
 'next_page_dict',
 'reported_financials',
 'reported_financials_dict',
 'swagger_types',
 'to_dict',
 'to_str']